In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')
train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')


In [ ]:
train.head()

In [ ]:
len(train),train.index.shape[-1]

In [ ]:
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
print("The Shape of the Dataset".format(),train.shape)
# Shape of the data

In [ ]:
good_reviews=train[train['target']==0]['question_text']
bad_reviews=train[train['target']==1]['question_text']
print("First 10 samples of good question text\n".format(),good_reviews[:10])
print("First 10 samples of bad question text\n".format(),bad_reviews[:10])

In [ ]:
#Count of good and bad reviews
count=train['target'].value_counts()
print('Total Counts of both sets'.format(),count)

print("==============")
#Creating a function to plot the counts using matplotlib
def plot_counts(count_good,count_bad):
    plt.rcParams['figure.figsize']=(6,6)
    plt.bar(0,count_good,width=0.6,label='Positive Reviews',color='Green')
    plt.legend()
    plt.bar(2,count_bad,width=0.6,label='Negative Reviews',color='Red')
    plt.legend()
    plt.ylabel('Count of Reviews')
    plt.xlabel('Types of Reviews')
    plt.show()
    
count_good=train[train['target']==0]
count_bad=train[train['target']==1]
plot_counts(len(count_good),len(count_bad))

In [ ]:
#Analyse the count of words in each segment- both positive and negative reviews


#Create generic plotter with Seaborn
def plot_count(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Red')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    




count_good_words=count_good['question_text'].str.split().apply(lambda z:len(z))
count_bad_words=count_bad['question_text'].str.split().apply(lambda z:len(z))
print("Positive Review Words:" + str(count_good_words))
print("Negative Review Words:" + str(count_bad_words))
plot_count(count_good_words,count_bad_words,"Positive Review","Negative Review","Reviews Word Analysis")

In [ ]:
#Count Punctuations/Stopwords/Codes and other semantic datatypes
#We will be using the "generic_plotter" function.

count_good_punctuations=count_good['question_text'].apply(lambda z: len([c for c in str(z) if c in string.punctuation]))
count_bad_punctuations=count_bad['question_text'].apply(lambda z:len([c for c in str(z) if c in string.punctuation]))
plot_count(count_good_punctuations,count_bad_punctuations,"Positive Review Punctuations","Negative Review Punctuations","Reviews Word Punctuation Analysis")

In [ ]:
count_good_punctuations.value_counts(1)

In [ ]:
#Analyse Stopwords

def plot_count_1(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Orange')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    


stops=set(stopwords.words('english'))
count_good_stops=count_good['question_text'].apply(lambda z : np.mean([len(z) if w in stops else 0 for w in str(z).split()]))
count_bad_stops=count_bad['question_text'].apply(lambda z : np.mean([len(z) if w in stops else 0 for w in str(z).split()]))
plot_count_1(count_good_stops,count_bad_stops,"Positive Question Stopwords","Negative Question Stopwords","Question Stopwords Analysis")

In [ ]:
count_bad_stops.value_counts(1)

In [ ]:
#WordCloud Visualizations
#Method for creating wordclouds

def display_cloud(data,color,figsize):
    plt.subplots(figsize=figsize)
    wc = WordCloud(stopwords=STOPWORDS,background_color="white", contour_width=2, contour_color=color,
                   max_words=2000, max_font_size=256,
                   random_state=42)
    wc.generate(' '.join(data))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis('off')
    plt.show()
    
display_cloud(train['question_text'],color='red',figsize=(15,15))

In [ ]:
display_cloud(count_good['question_text'],color='red',figsize=(15,15))

In [ ]:
display_cloud(count_bad['question_text'],'blue',figsize=(15,15))

In [ ]:
#Simplified counter function
def create_corpus(word):
    corpus=[]
    
    for x in train[train['target']==word]['question_text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

corpus=create_corpus(1)
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:100]:
    if (word not in stops) :
        x.append(word)
        y.append(count)
sns.barplot(x=y,y=x)

Inference From Analysis -I
The following can be inferred from the data:

The dataset is unbalanced.
The dataset contains unequal number of semantics for reviews of both polarity.
The dataset doesn't contains redundant words and html syntaxes.
Punctuations/stopwords are present in a equal distribution in the dataset.
This tells us that we have to do lots of cleaning!

In [ ]:
#Gram analysis on Training set- Bigram and Trigram
stopword=set(stopwords.words('english'))
def gram_analysis(data,gram):
    tokens=[t for t in data.lower().split(" ") if t!="" if t not in stopword]
    ngrams=zip(*[tokens[i:] for i in range(gram)])
    final_tokens=[" ".join(z) for z in ngrams]
    return final_tokens


#Create frequency grams for analysis
    
def create_dict(data,grams):
    freq_dict=defaultdict(int)
    for sentence in data:
        for tokens in gram_analysis(sentence,grams):
            freq_dict[tokens]+=1
    return freq_dict

def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["n_gram_words"].values[::-1],
        x=df["n_gram_frequency"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace



def create_new_df(freq_dict,):
    freq_df=pd.DataFrame(sorted(freq_dict.items(),key=lambda z:z[1])[::-1])
    freq_df.columns=['n_gram_words','n_gram_frequency']
    #print(freq_df.head())
    #plt.barh(freq_df['n_gram_words'][:20],freq_df['n_gram_frequency'][:20],linewidth=0.3)
    #plt.show()
    trace=horizontal_bar_chart(freq_df[:20],'orange')
    return trace
    
def plot_grams(trace_zero,trace_one):
    fig = subplots.make_subplots(rows=1, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words of positive Questions", 
                                          "Frequent words of negative Questions"])
    fig.append_trace(trace_zero, 1, 1)
    fig.append_trace(trace_ones, 1, 2)
    fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
    py.iplot(fig, filename='word-plots')
    
    
train_df_zero=count_bad['question_text']
train_df_ones=count_good['question_text']

print("Bi-gram analysis")
freq_train_df_zero=create_dict(train_df_zero[:200],2)
#print(freq_train_df_zero)
trace_zero=create_new_df(freq_train_df_zero)
freq_train_df_ones=create_dict(train_df_ones[:200],2)
#print(freq_train_df_zero)
trace_ones=create_new_df(freq_train_df_ones)
plot_grams(trace_zero,trace_ones)
print("Tri-gram analysis")
freq_train_df_zero=create_dict(train_df_zero[:200],3)
#print(freq_train_df_zero)
trace_zero=create_new_df(freq_train_df_zero)
freq_train_df_ones=create_dict(train_df_ones[:200],3)
#print(freq_train_df_zero)
trace_ones=create_new_df(freq_train_df_ones)
plot_grams(trace_zero,trace_ones)

Inference from Analysis - II
In this section, we have analysed based on positional features of words in a corpus/sentence/paragraph. The Gram analysis,particularly the pentagram analysis provides an idea which sentences occur more often in the corpus. And in most of the cases, these bag of words are the ones picked up by any frequency vectorization technique.

Thus this provides an outline as to the frequency of the conjuction of words which are occuring at the highest frequency. Another important aspect is that,there is a presence of certain html tags and punctuations which have to be removed as these are adding noise to the review corpus. This will be taken up in the cleaning phase.

Time for some Cleaning!
Before we move ahead , let us clean the dataset and remove the redundancies.This includes


- Emojis
- Stopwords
- Punctuations
- Expanding Abbreviations

These will be sufficient for cleaning the corpus!

Regex is a very good tool which will help us to do this cleaning.

In [ ]:
class TextCleaningUtils:
    '''
        This class contains implementations of various text cleaning operations (Static Methods)
    '''

    cleaning_regex_map = {
        'web_links': r'(?i)(?:(?:http(?:s)?:)|(?:www\.))\S+',
#         
        'special_chars': r'[^a-zA-Z\s\.,!?;:]+',  ## removing nums
        'redundant_spaces': r'\s\s+',
        'redundant_newlines': r'[\r|\n|\r\n]+',
        'twitter_handles': r'[#@]\S+',
        'punctuations': r'[\.,!?;:]+'
    }
    
    def expand_abbreviations(text):
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"there's", "there is", text)
        text = re.sub(r"We're", "We are", text)
        text = re.sub(r"That's", "That is", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"they're", "they are", text)
        text = re.sub(r"Can't", "Cannot", text)
        text = re.sub(r"wasn't", "was not", text)
        text = re.sub(r"don\x89Ûªt", "do not", text)
        text= re.sub(r"aren't", "are not", text)
        text = re.sub(r"isn't", "is not", text)
        text = re.sub(r"What's", "What is", text)
        text = re.sub(r"haven't", "have not", text)
        text = re.sub(r"hasn't", "has not", text)
        text = re.sub(r"There's", "There is", text)
        text = re.sub(r"He's", "He is", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"You're", "You are", text)
        text = re.sub(r"I'M", "I am", text)
        text = re.sub(r"shouldn't", "should not", text)
        text = re.sub(r"wouldn't", "would not", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"i'm", "I am", text)
        text = re.sub(r"I\x89Ûªm", "I am", text)
        text = re.sub(r"I'm", "I am", text)
        text = re.sub(r"Isn't", "is not", text)
        text = re.sub(r"Here's", "Here is", text)
        text = re.sub(r"you've", "you have", text)
        text = re.sub(r"you\x89Ûªve", "you have", text)
        text = re.sub(r"we're", "we are", text)
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"we've", "we have", text)
        text = re.sub(r"it\x89Ûªs", "it is", text)
        text = re.sub(r"doesn\x89Ûªt", "does not", text)
        text = re.sub(r"It\x89Ûªs", "It is", text)
        text = re.sub(r"Here\x89Ûªs", "Here is", text)
        text = re.sub(r"who's", "who is", text)
        text = re.sub(r"I\x89Ûªve", "I have", text)
        text = re.sub(r"y'all", "you all", text)
        text = re.sub(r"can\x89Ûªt", "cannot", text)
        text = re.sub(r"would've", "would have", text)
        text = re.sub(r"it'll", "it will", text)
        text = re.sub(r"we'll", "we will", text)
        text = re.sub(r"wouldn\x89Ûªt", "would not", text)
        text = re.sub(r"We've", "We have", text)
        text = re.sub(r"he'll", "he will", text)
        text = re.sub(r"Y'all", "You all", text)
        text = re.sub(r"Weren't", "Were not", text)
        text = re.sub(r"Didn't", "Did not", text)
        text = re.sub(r"they'll", "they will", text)
        text = re.sub(r"DON'T", "DO NOT", text)
        text = re.sub(r"That\x89Ûªs", "That is", text)
        text = re.sub(r"they've", "they have", text)
        text = re.sub(r"they'd", "they would", text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"should've", "should have", text)
        text = re.sub(r"You\x89Ûªre", "You are", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"Don\x89Ûªt", "Do not", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"weren't", "were not", text)
        text = re.sub(r"They're", "They are", text)
        text = re.sub(r"Can\x89Ûªt", "Cannot", text)
        text = re.sub(r"you\x89Ûªll", "you will", text)
        text = re.sub(r"I\x89Ûªd", "I would", text)
        text = re.sub(r"let's", "let us", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"you're", "you are", text)
        text = re.sub(r"i've", "I have", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"doesn't", "does not",text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"didn't", "did not", text)
        text = re.sub(r"ain't", "am not", text)
        text = re.sub(r"you'll", "you will", text)
        text = re.sub(r"I've", "I have", text)
        text = re.sub(r"Don't", "do not", text)
        text = re.sub(r"I'll", "I will", text)
        text = re.sub(r"I'd", "I would", text)
        text = re.sub(r"Let's", "Let us", text)
        text = re.sub(r"you'd", "You would", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"Ain't", "am not", text)
        text = re.sub(r"Haven't", "Have not", text)
        text = re.sub(r"Hadn't", "Had not", text)
        text = re.sub(r"Could've", "Could have", text)
        text = re.sub(r"youve", "you have", text)  
        text = re.sub(r"donå«t", "do not", text)  

        return text
    
    def remove_emojis(text):
        emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text=emoji_clean.sub(r'',text)
        url_clean= re.compile(r"https://\S+|www\.\S+")
        text=url_clean.sub(r'',text)
        return text
    


    @staticmethod
    def clean_text_from_regex(text, text_clean_regex):
        '''
            Follow a particular cleaning expression, provided
            as an input by an user to clean the text.
        '''

        text = text_clean_regex.sub(' ', text).strip()
        return text

    @staticmethod
    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    @staticmethod
    def remove_special_chars(text):
        '''
            Replace any special character provided as default,
            which is present in the text with space
        '''

        special_chars_regex = re.compile(TextCleaningUtils.cleaning_regex_map['special_chars'])
        text = TextCleaningUtils.clean_text_from_regex(text, special_chars_regex)
        return text

    @staticmethod
    def remove_redundant_spaces(text):
        '''
            Remove any redundant space provided as default,
            that is present in the text.
        '''

        redundant_spaces_regex = re.compile(
            TextCleaningUtils.cleaning_regex_map['redundant_spaces'])
        text = TextCleaningUtils.clean_text_from_regex(text, redundant_spaces_regex)
        return text

    @staticmethod
    def remove_web_links(text):
        '''
            Removes any web link that follows a particular default expression,
            present in the text.
        '''

        web_links_regex = re.compile(TextCleaningUtils.cleaning_regex_map['web_links'])
        text = TextCleaningUtils.clean_text_from_regex(text, web_links_regex)
        return text

    @staticmethod
    def remove_twitter_handles(text):
        '''
            Removes any twitter handle present in the text.
        '''

        twitter_handles_regex = re.compile(TextCleaningUtils.cleaning_regex_map['twitter_handles'])
        text = TextCleaningUtils.clean_text_from_regex(text, twitter_handles_regex)
        return text

    @staticmethod
    def remove_redundant_newlines(text):
        '''
            Removes any redundant new line present in the text.
        '''

        redundant_newlines_regex = re.compile(
            TextCleaningUtils.cleaning_regex_map['redundant_newlines'])
        text = TextCleaningUtils.clean_text_from_regex(text, redundant_newlines_regex)
        return text

    @staticmethod
    def remove_punctuations(text):
        '''
            Removes any punctuation that follows the default expression, in the text.
        '''

        remove_punctuations_regex = re.compile(TextCleaningUtils.cleaning_regex_map['punctuations'])
        text = TextCleaningUtils.clean_text_from_regex(text, remove_punctuations_regex)
        return text

    @staticmethod
    def remove_exaggerated_words(text):
        '''
            Removes any exaggerated word present in the text.
        '''

        return ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))

    @staticmethod
    def replace_multiple_chars(text):
        '''
            Replaces multiple characters present in the text.
        '''

        char_list = ['.', '?', '!', '#', '$', '/', '@', '*', '(', ')', '+']
        final_text = ''
        for i in char_list:
            if i in text:
                pattern = "\\" + i + '{2,}'
                repl_str = i.replace("\\", "")
                text = re.sub(pattern, repl_str, text)
                final_text = ' '.join(text.split())
        return final_text

    @staticmethod
    def replace_sign(text):
        '''
            Replaces any sign with words like & with 'and', in the text.
        '''
        sign_list = {'&': ' and ', '/': ' or ', '\xa0': ' '}
        final_text = ''
        for i in sign_list:
            if i in text:
                text = re.sub(i, sign_list[i], text)
                final_text = ' '.join(text.split())
        return final_text

    @staticmethod
    def remove_accented_char(text):
        text = unicodedata.normalize('NFD', text) \
            .encode('ascii', 'ignore') \
            .decode("utf-8")
        return str(text)

    @staticmethod
    def replace_characters(text, replace_map):
        '''
            Replaces any character custom provided by an user.
        '''

        for char, replace_val in replace_map.items():
            text = text.replace(char, replace_val)
        return text
    
def clean_data(df,col_to_clean):
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.expand_abbreviations)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_emojis)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_special_chars)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_redundant_spaces)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_punctuations)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_exaggerated_words)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_redundant_newlines)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_twitter_handles)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_web_links)
    df[col_to_clean] = df[col_to_clean].astype(str)
    df[col_to_clean] = df[col_to_clean].str.lower()
    
    return (df)

In [ ]:
import itertools
from bs4 import BeautifulSoup

In [ ]:
count_good_df = clean_data(count_good, 'question_text')

In [ ]:
count_bad_df = clean_data(count_bad, 'question_text')

In [ ]:
train_df = clean_data(train, 'question_text')

In [ ]:
train_df['target'].value_counts(1)

In [ ]:
count_bad

In [ ]:
#Apply Gram Analysis
train_df_zero=count_bad_df['question_text']
train_df_ones=count_good_df['question_text']
print("Tri-gram analysis")
freq_train_df_zero=create_dict(train_df_zero[:200],3)
#print(freq_train_df_zero)
trace_zero=create_new_df(freq_train_df_zero)
freq_train_df_ones=create_dict(train_df_ones[:200],3)
#print(freq_train_df_zero)
trace_ones=create_new_df(freq_train_df_ones)
plot_grams(trace_zero,trace_ones)

In [ ]:
print("Penta-gram analysis")
freq_train_df_zero=create_dict(train_df_zero[:200],5)
#print(freq_train_df_zero)
trace_zero=create_new_df(freq_train_df_zero)
freq_train_df_ones=create_dict(train_df_ones[:200],5)
#print(freq_train_df_zero)
trace_ones=create_new_df(freq_train_df_ones)
plot_grams(trace_zero,trace_ones)

In [ ]:
display_cloud(train_df['question_text'],color='red',figsize=(15,15))

In [ ]:
display_cloud(count_bad_df['question_text'],color='red',figsize=(15,15))

In [ ]:
display_cloud(count_good_df['question_text'],color='red',figsize=(15,15))

In [ ]:
#Lemmatize the dataset
from nltk.stem import WordNetLemmatizer


def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data

train_df['question_text'] = train_df['question_text'].apply(lambda z: lemma_traincorpus(z))

In [ ]:
#For example let us try to stem them and check  a sample

from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data

sample_train_df=train_df[5:10]
sample_train_df['question_text']=sample_train_df['question_text'].apply(lambda z: stem_traincorpus(z))
sample_train_df['question_text']

In [ ]:
%%time
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,3))
train_tfidf=tfidf_vect.fit_transform(train_df['question_text'].values.tolist())
train_tfidf.shape

In [ ]:
## Outputs from the TF-IDF transformed data
print(train_tfidf)

In [ ]:
#Count Vectorization
import matplotlib
import matplotlib.pyplot as plt
def vectorize(data):
    cv=CountVectorizer()
    fit_data_cv=cv.fit_transform(data)
    return fit_data_cv,cv

#Tfidf vectorization from sklearn
def tfidf(data):
    tfidfv=TfidfVectorizer()
    fit_data_tfidf=tfidfv.fit_transform(data)
    return fit_data_cv,tfidfv

def dimen_reduc_plot(test_data,test_label,option):
    tsvd= TruncatedSVD(n_components=2,algorithm="randomized",random_state=42)
    tsne=TSNE(n_components=2,random_state=42) #not recommended instead use PCA
    pca=SparsePCA(n_components=2,random_state=42)
    if(option==1):
        tsvd_result=tsvd.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        
        sns.scatterplot(x=tsvd_result[:,0],y=tsvd_result[:,1],hue=test_label        )
        
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(tsvd_result[:,0],tsvd_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Insincere Question')
        color_orange=mpatches.Patch(color='orange',label='Sincere Question')
        plt.legend(handles=[color_orange,color_red])
        plt.title("TSVD")
        plt.show()
    if(option==2):
        tsne_result=tsne.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=tsne_result[:,0],y=tsne_result[:,1],hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=tsne_result[:,0],y=tsne_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Insincere Question')
        color_orange=mpatches.Patch(color='orange',label='Sincere Question')
        plt.legend(handles=[color_orange,color_red])
        plt.title("PCA")
        plt.show() 
    if(option==3):
        pca_result=pca.fit_transform(test_data.toarray())
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=pca_result[:,0],y=pca_result[:,1],hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=pca_result[:,0],y=pca_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Insincere Question')
        color_orange=mpatches.Patch(color='orange',label='Sincere Question')
        plt.legend(handles=[color_orange,color_red])
        plt.title("TSNE")
        plt.show()
train_data=train_df       
data_vect=train_data['question_text'].values
data_vect_good=count_good['question_text'].values
target_vect=train_data['target'].values
target_data_vect_good=train_df[train_df['target']==0]['target'].values
data_vect_bad=count_bad['question_text'].values
target_data_vect_bad=train_df[train_df['target']==1]['target'].values
train_data_cv,cv= vectorize(data_vect)
real_review_train_data_cv,cv=vectorize(data_vect_good)

print(train_data.head())
dimen_reduc_plot(train_data_cv,target_vect,1)
dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,1)
#dimen_reduc_plot(real_review_train_data_cv,target_data_vect_bad,1)
# dimen_reduc_plot(train_data_cv,target_vect,3)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,3)
# dimen_reduc_plot(train_data_cv,target_vect,2)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,2)

In [ ]:
check_df=list(train_df['question_text'].str.split())

In [ ]:
%%time
## Load word2vec algorithm from gensim
from gensim.models import Word2Vec,KeyedVectors

model=Word2Vec(check_df,min_count=1)
word_li=list(model.wv.vocab)
print(word_li[:5])

In [ ]:
#View the Tensor
print(model)
print(model['questions'])

In [ ]:
#View the Embedding Word Vector
plt.plot(model['questions'])
plt.show()

In [ ]:
##save the modeled words produced from Word2Vec
model.save('word2vec_model.bin')
loaded_model=KeyedVectors.load('word2vec_model.bin')
print(loaded_model)

In [ ]:
#Measure Cosine distance
distance=model.similarity('questions','insincere')
print(distance)

In [ ]:
# PCA transform in 2D for visualization of embedded words
from matplotlib import pyplot
pca = PCA(n_components=2)
transformation_model=loaded_model[loaded_model.wv.vocab]
result = pca.fit_transform(transformation_model[:50])
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(loaded_model.wv.vocab)
for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

In [ ]:
#Using Google News Embeddings for the corpus
from gensim.models import Word2Vec

#download Google News Embeddings
google_news_embed="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

google_loaded_model=KeyedVectors.load_word2vec_format(google_news_embed,binary=True)
print(google_loaded_model)

In [ ]:
plt.bar([i for i in range(0,300)], [i for i in google_loaded_model['questions']])

In [ ]:
#Visualize the Word Vectors
##Comparing the similarities of words graphically
plt.figure(figsize=(20,9))
plt.bar([i for i in range(0,300)], [i for i in google_loaded_model['questions']])
plt.bar([i for i in range(0,300)], [i for i in google_loaded_model['insincere']])
plt.show()

In [ ]:
# PCA transform in 2D for visualization of google news embedded words
from matplotlib import pyplot

plt.figure(figsize=(20,12))
pca = PCA(n_components=2)
transformation_model = google_loaded_model[google_loaded_model.vocab]
result = pca.fit_transform(transformation_model[:100])
# create a scatter plot of the projection
plt.scatter(result[:, 0], result[:, 1])
words = list(loaded_model.wv.vocab)
for i, word in enumerate(words[:100]):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
    plt.title("PCA Transformation of word vectors with Google News Embeddings on to 2D-space\n")
plt.show()

Glove Embeddings

* Glove embeddings rely on global vector representations mechanism, which is an unsupervised algorithm.

* This captures both the global corpus statistics as well as local semantic information.

* Glove vectors used here can be converted from "txt" format to Word2Vec format by using scripts provided in the Gensim library.

* This allows us to manipulate the glove embeddings in a manner similar to Word2Vec and apply the similarity metric.

* The loss function for the glove relies on logistic regression of the log co-occurence probabilities.

In [ ]:

'''
import zipfile

path_to_zip_file = '../input/quora-insincere-questions-classification/embeddings.zip'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall('./')'''

In [ ]:
'''#Using Google News Embeddings For our corpus
google_news_embed = './GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
google_loaded_model=KeyedVectors.load_word2vec_format(google_news_embed,binary=True)
print(google_loaded_model)'''

In [ ]:
'''#Visualize the Word Vectors
plt.plot(google_loaded_model['questions'])
plt.plot(google_loaded_model['insincere'])
plt.show()'''

In [ ]:
'''# PCA transform in 2D for visualization of google news embedded words
from matplotlib import pyplot
pca = PCA(n_components=2)
transformation_model=google_loaded_model[google_loaded_model.wv.vocab]
result = pca.fit_transform(transformation_model[:50])
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(google_loaded_model.wv.vocab)
for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()'''

In [ ]:
#%env JOBLIB_TEMP_FOLDER=/tmp

from gensim.scripts.glove2word2vec import glove2word2vec

glove_file='../input/glove6b50dtxt/glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.txt'
glove_loaded=glove2word2vec(glove_file, word2vec_output_file)
print(glove_loaded)

In [ ]:
#Comparing the similarities of words graphically
plt.figure(figsize=(20,8))
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
plt.bar([i for i in range(0,50)], [i for i in glove_model['questions']])
plt.bar([i for i in range(0,50)], [i for i in glove_model['insincere']])
plt.title("Visualize the Word Vectors with GloVe Embeddings")
plt.show()

In [ ]:
#PCA transform in 2D for visualization of glove embedded words

plt.figure(figsize=(20,15))
pca = PCA(n_components=2)
transformation_model = glove_model[glove_model.vocab]
result = pca.fit_transform(transformation_model[:100])
# create a scatter plot of the projection
plt.scatter(result[:, 0], result[:, 1])
words = list(loaded_model.wv.vocab)
for i, word in enumerate(words[:100]):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
    plt.title("PCA Transformation of word vectors with GloVe Embeddings on to 2D-space\n")
plt.show()

## **Fastext embeddings**

In [ ]:
#Using the fasttext word embeddding from crawl

from gensim.models import Word2Vec,KeyedVectors

fasttext_file="../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"

In [ ]:
plt.figure(figsize=(20,8))

fasttext_model = KeyedVectors.load_word2vec_format(fasttext_file, binary=False)

y1 = [i for i in fasttext_model['questions']]
y2 = [i for i in fasttext_model['insincere']]

plt.bar([i for i in range(0,len(y1))], y1)
plt.bar([i for i in range(0,len(y2))], y2, alpha = 0.3)
plt.title("Visualize the Word Vectors with Fast Text Embeddings")
plt.show()

In [ ]:
#PCA transform in 2D for visualization of glove embedded words

pca = PCA(n_components=2)
transformation_model=fasttext_model[fasttext_model.vocab]
result = pca.fit_transform(transformation_model[:50])

plt.figure(figsize=(20,12))
#create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(fasttext_model.vocab)
for i, word in enumerate(words[:50]):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
    plt.title('PCA Transformation of word vectors with Fast Text Embeddings on to 2D-space\n')
plt.show()

### Procedure to create Embedding Matrix

The importance of building a Embedding matrix is to have mutual co-occurence embedding probabilities (vectors) of all the words present in the corpus. This are the formats wherein neural network frameworks like Keras,Tensorflow are likely to be used.

* These embedding vectors are then passed through deep learning layers.

* The flexibility of these static embeddings is that they are able to provide a good benchmark on the initial task (classification) in our case.

* These also provide a very good approximation to the amount of percentage accuracy or loss which can be achieved with a coupled Encoder-Decoder/Transformer like architectures.

We will be using the Keras-Tensorflow framework to build the embedding matrix.

In [ ]:
#copy of the data with the first column dropped
# new_train_data = training_data.drop(columns='qid')[:10000].copy()

sample_train_df = train_df.set_index('qid').sample(10000)

In [ ]:
%%time
#Creating Embedding Matrix
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

max_len=500
max_features=5000 
embed_size=300

train_sample = sample_train_df['question_text']



In [ ]:
##Tokenizing steps
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sample))
train_sample=tokenizer.texts_to_sequences(train_sample)



In [ ]:
#Pad the sequence- To allow same length for all vectorized words
train_sample=pad_sequences(train_sample,maxlen=max_len)


EMBEDDING_FILE = '../input/wikinews300d1msubwordvec/wiki-news-300d-1M-subword.vec'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
